In [3]:
import numpy as np
from qiskit import *
# from qiskit_machine_learning.algorithms.classifiers import VQC
# from qiskit import BasicAer
# from qiskit.utils import QuantumInstance, algorithm_globals
# from qiskit.algorithms.optimizers import SPSA
# from qiskit.circuit.library import TwoLocal, ZZFeatureMap
# from qiskit_machine_learning.datasets import ad_hoc_data


In [11]:
q_reg = QuantumRegister(4)
c_reg = ClassicalRegister(1)

cirq = QuantumCircuit(q_reg, c_reg)
cirq.ry(2, 0)

cirq.measure(q_reg[0], c_reg)


# Reading Output

After applying the feature map and executing the parametrized circuit, the probability that the measurement of the qubit results in the state $\ket{1}$ is expressed as $$\sum_{k=2^{n-1} + 1}^{2^n} |(U_\theta (\phi(x)))_k|^2$$
To resolve the probabilitic problem here, we may 'sample' the circuit some $S$ times to estimate $p(q_0=1)$ from $s_1, \cdots s_S$

In [21]:
# Read out input 

from concurrent.futures import process

# Measure and process output
def process_output(circuit, quantum_reg, classical_reg):
    s = 100 # arbitrary S, will shift to match output later
    arr = []

    simulator = Aer.get_backend('aer_simulator')

    job = execute(circuit, simulator, shots=s)
    result = job.result()
    counts = result.get_counts(circuit)
    return counts

# Return probabilities
def probabilities(counts, labels, shots=100):
    result_dict = {labels[0]: 0, labels[1]: 0}
    for key, value in counts.items():
        label_val = key[-1]
        if int(label_val) == 0:
            result_dict[labels[0]] = value / shots
        if int(label_val) == 1:
            result_dict[labels[1]] = value / shots

    return result_dict

def data_dictionary(parameters, x, feature_map, var_form):
    p = {}
    for i, param in enumerate(feature_map.ordered_parameters):
        p[param] = x[i]
    for i, param in enumerate(var_form.ordered_parameters):
        p[param] = parameters[i]
    return p

def classification(x_array, parameters, labels, statevector, circuit):
    circs = []
    prob = []
    for x in x_array:
        circ = circuit.assign_parameters(data_dictionary(parameters, x))
        cirq = statevector.evolve(circ)
        circs.append(cirq)
    for q in circs:
        counts = q.to_counts()
        temp = probabilities(counts, labels)
        prob.append(temp)
    return prob

print(probabilities(process_output(cirq, q_reg, c_reg), [0, 1], 100))

{0: 0.21, 1: 0.79}


# Calculating a Gradient

As described by Schuld et. al in *Circuit-centric quantum classifiers*, we can compute a gradient for the function we are to optimize with something known as the parameter shift rule. Further described by Crooks in *Gradients of parameterized quantum gates using the parameter-shift rule and gate decomposition*, we can express this rule as 
$$\frac{d}{d\theta} f(\theta) = r\left[f\left(\theta + \frac{\pi}{4r}\right) - f\left(\theta - \frac{\pi}{4r}\right) \right]$$
where the shift constant $r = \frac{a}{2} (e_1 - e_0)$ and $e_1,\ e_0$ are eigenvalues of the unitary gate $G$.

# Cost Function
In actual practice, we must define a cost function that will define how "well" our proposed parameters are doing. 
This can be expressed as (according to Rodney Osodo) 
$$L = \frac{1}{N} \sum_{i=1}^n (y_i^{truth} - f(\omega, b, x_i))^2$$
We can use defined optimization algorithms like COBYLA to then optimize the parameters.

In [22]:
def mse(probabilities, expected):
    probability = probabilities[expected]
    return (1 - probability) ** 2

def cost(X, Y, labels, parameters, statevector, shots=100):
    cost = 0
    training_labels = []
    training_samples = []
    for sample in X:
        training_samples.append(sample)

    for label in Y:
        if label == 0:
            training_labels.append(labels[0])
        else:
            training_labels.append(labels[1])

    prob = classification(training_samples, parameters, labels)

    for i in range(len(prob)):
        cost += mse(prob[i], training_labels[i])
    
    cost /= len(training_samples)

    return cost

    